In [1]:
import tensorflow as tf
print(tf.__version__)
import collections
import cv2
import numpy as np

2.1.0


In [2]:
THRESHOLD = 0.5

def filter(x, y, a=3, b=6):
    keep = (y == a) | (y == b)
    x, y = x[keep], y[keep]
    y = y == a
    return x,y


def remove_ambiguous(xs, ys, a=3, b=6):
    mapping = collections.defaultdict(set)
    # Determine the set of labels for each unique image:
    for x,y in zip(xs,ys):
       mapping[tuple(x.flatten())].add(y)
    
    new_x = []
    new_y = []
    for x,y in zip(xs, ys):
      labels = mapping[tuple(x.flatten())]
      if len(labels) == 1:
          new_x.append(x)
          new_y.append(list(labels)[0])
      else:
          # Throw out images that match more than one label.
          pass
    
    num_a = sum(1 for value in mapping.values() if True in value)
    num_b = sum(1 for value in mapping.values() if False in value)
    num_both = sum(1 for value in mapping.values() if len(value) == 2)

    print("Number of unique images:", len(mapping.values()))
    print("Number of {}s: ".format(a), num_a)
    print("Number of {}s: ".format(b), num_b)
    print("Number of contradictory images: ", num_both)
    print("Initial number of examples: ", len(xs))
    print("Remaining non-contradictory examples: ", len(new_x))
    print()
    
    return np.array(new_x), np.array(new_y)


def get_edge(num_qubits):
    h = int(np.sqrt(num_qubits))
    assert h**2 == num_qubits, 'num_qubits is not a perfect square of an integer!'
    return h

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [5]:
x, y = x_train, y_train
x = x[..., np.newaxis] / 255.0
x, y = filter(x, y, a=3, b=6)

In [6]:
x_small = tf.image.resize(x, (4, 4)).numpy()
x_bin = np.array(x_small > THRESHOLD, dtype=np.float32)
x_nocon, y_nocon = remove_ambiguous(x_bin, y, a=3, b=6)

Number of unique images: 193
Number of 3s:  125
Number of 6s:  113
Number of contradictory images:  45
Initial number of examples:  12049
Remaining non-contradictory examples:  3649



In [7]:
num_a = sum(y_nocon)
num_b = 3649 - num_a
print(num_a)
print(num_b)

2074
1575


In [23]:
x, y = x_test, y_test
x = x[..., np.newaxis] / 255.0
x, y = filter(x, y, a=3, b=6)

In [9]:
x_small = tf.image.resize(x, (4, 4)).numpy()
x_bin = np.array(x_small > THRESHOLD, dtype=np.float32)
x_nocon, y_nocon = remove_ambiguous(x_bin, y, a=3, b=6)

Number of unique images: 115
Number of 3s:  65
Number of 6s:  72
Number of contradictory images:  22
Initial number of examples:  1968
Remaining non-contradictory examples:  890



In [22]:
num_a = sum(y_nocon)
num_b = 890 - num_a
print(num_a)
print(num_b)

332
558


In [20]:
import os
import matplotlib.pyplot as plt
import numpy as np

import matplotlib.font_manager as font_manager

path = '/home/ndong/Desktop/ubuntu-font-family/Ubuntu-R.ttf'
prop = font_manager.FontProperties(fname=path, size=36)

def line_plot(series, output_path, series_names=['DNN', 'QNN'], title=None,
    ymin=0, ymax=1, legend_loc="best", legend_ncols=1,
    x_label="Number of Batches", y_label=None):
    """
    Plot average curves with uncertainty.
    series: list of [num_x, num_runs+1] np.array. num_runs are averaged
    and std. The first column is x, the rest are y points.
    """
    line_colors = [
        'b', # blue
        'r'  # red
        ]
    shade_colors = [
        "#6fbcf7", # light blue  
        "#ffcccb"  # light red
        ]

    plt.figure(figsize=(10,8))
    plt.style.use('ggplot')

    for i, s in enumerate(series):
        x = 10 * np.array(list(range(1, s.shape[0]+1)))
        y = np.mean(s, axis=1)
        std = np.std(s, axis=1)
        plt.plot(x, y, color=line_colors[i], label=series_names[i])
        plt.fill_between(x, y-std, y+std, alpha=0.5, facecolor=shade_colors[i], 
            linewidth=0)

    plt.ylabel(y_label, fontsize=24)
    plt.xlabel(x_label, fontsize=24)
    
    plt.xlim(0, s.shape[0]*10)
    if ymax is not None:
        plt.gca().set_ylim(top=ymax)
    if ymin is not None:
        plt.gca().set_ylim(bottom=ymin)

    plt.tick_params(labelsize=18)
    plt.xticks()

    plt.legend(loc=legend_loc)
    plt.legend(fontsize=24)

    plt.title(title, fontsize=36, fontproperties=prop)

    plt.savefig(output_path, bbox_inches='tight')
    plt.close()
    print('Saved to', output_path)    


if __name__ == '__main__':
    DNN = []
    for i in range(1, 6):
        dnn = np.load('/home/ndong/Desktop/QDA/Quantum/dnn_{}.npy'.format(i))
        DNN.append(dnn)

    DNN = np.array(DNN)
    DNN = DNN.transpose([1, 2, 0])
    dnn_train_loss = DNN[0]
    dnn_train_acc = DNN[1]
    dnn_test_acc = DNN[2]

    QNN = []
    for i in range(1, 4):
        qnn = np.load('/home/ndong/Desktop/QDA/Quantum/qnn_{}.npy'.format(i))
        QNN.append(qnn)

    QNN = np.array(QNN)
    QNN = QNN.transpose([1, 2, 0])
    qnn_train_loss = QNN[0]
    qnn_train_acc = QNN[1]
    qnn_test_acc = QNN[2]

    line_plot([dnn_train_loss, qnn_train_loss], '/home/ndong/Desktop/QDA/train_loss.png', 
        title="Train Loss", ymax=None)

    line_plot([dnn_train_acc, qnn_train_acc], '/home/ndong/Desktop/QDA/train_acc.png', 
        title="Train Accuracy")

    line_plot([dnn_test_acc, qnn_test_acc], '/home/ndong/Desktop/QDA/test_acc.png', 
        title="Test Accuracy")

Saved to /home/ndong/Desktop/QDA/train_loss.png
Saved to /home/ndong/Desktop/QDA/train_acc.png
Saved to /home/ndong/Desktop/QDA/test_acc.png


In [21]:
if __name__ == '__main__':
    DNN = []
    for i in range(1, 6):
        dnn = np.load('/home/ndong/Desktop/QDA/Quantum/dnn_da_{}.npy'.format(i))
        DNN.append(dnn)

    DNN = np.array(DNN)
    DNN = DNN.transpose([1, 2, 0])
    dnn_train_loss = DNN[0]
    dnn_train_acc = DNN[1]
    dnn_test_acc = DNN[2]

    QNN = []
    for i in range(1, 4):
        qnn = np.load('/home/ndong/Desktop/QDA/Quantum/qnn_da_{}.npy'.format(i))
        QNN.append(qnn)

    QNN = np.array(QNN)
    QNN = QNN.transpose([1, 2, 0])
    qnn_train_loss = QNN[0]
    qnn_train_acc = QNN[1]
    qnn_test_acc = QNN[2]

    line_plot([dnn_train_loss, qnn_train_loss], '/home/ndong/Desktop/QDA/da_train_loss.png', 
        title="Train Loss", ymax=None)

    line_plot([dnn_train_acc, qnn_train_acc], '/home/ndong/Desktop/QDA/da_train_acc.png', 
        title="Train Accuracy")

    line_plot([dnn_test_acc, qnn_test_acc], '/home/ndong/Desktop/QDA/da_test_acc.png', 
        title="Test Accuracy")

Saved to /home/ndong/Desktop/QDA/da_train_loss.png
Saved to /home/ndong/Desktop/QDA/da_train_acc.png
Saved to /home/ndong/Desktop/QDA/da_test_acc.png


In [4]:
def visualize(imgs, arr = (2, 4), invert=False):
    if invert:
        imgs = 255 - imgs
    imgs =  imgs[..., np.newaxis]
    imgs = tf.image.resize(imgs, (28 * 3, 28 * 3)).numpy()
    imgs = np.squeeze(imgs, axis=3)
    num = len(imgs)
    assert  num == arr[0] * arr[1]
    COL = []
    for i in range(arr[0]):
        ROW = []
        for j in range(arr[1]):
            img = imgs[i * arr[1] +j]
            ROW.append(img)
        row = np.concatenate(ROW, axis=1)
        COL.append(row)
    col = np.concatenate(COL, axis=0)
    return col

In [5]:
x, y = filter(x_test, y_test, a=3, b=6)


In [10]:
img = visualize(x[:8])
cv2.imwrite('/home/ndong/Desktop/QDA/source.png', img)

True

In [12]:
img = visualize(x[8:16], invert=True)
cv2.imwrite('/home/ndong/Desktop/QDA/target.png', img)

True

In [8]:
336 / 28

12.0